In [1]:
# add parent dir to sys path for import of modules
import os
import sys

# find recursively the project root dir
parent_dir = str(os.getcwdb())
while not os.path.exists(os.path.join(parent_dir, "README.md")):
    parent_dir = os.path.abspath(os.path.join(parent_dir, os.pardir))
sys.path.insert(0, parent_dir)

In [2]:
import json
import pandas as pd

from utils import ROOT_DIR, load_pickle
from PetReader import pet_reader

INFO:Utilities:Loaded config: {'general-seed': 42, 'keywords-filtered-approach': {'bert-model-name': 'distilbert-base-uncased', 'label-set': 'all', 'label-number': 9, 'other-labels-weight': 0.1, 'num-labels': 9}, 'same-gateway-classifier': {'context_label_length': 350}, 'synonym-samples-start-number': 500}
INFO:PetReader:Reload pet_reader from C:\Users\janek\Development\Git\master-thesis\data/other/pet_reader.pkl


In [3]:
with open(os.path.join(ROOT_DIR, "data/results/keywords_filtered/key_words_literature_tc_filtered_[all_0.1_og]_fixed/filtering_XOR Gateways.json"), "r") as file:
    filtering_file = json.load(file)

In [19]:
KEPT = 'kept'
CORRECTLY_KEPT = 'correctly_kept'
WRONGLY_KEPT = 'wrongly_kept'
DROPPED = 'dropped'
CORRECTLY_DROPPED = 'correctly_dropped'
WRONGLY_DROPPED = 'wrongly_dropped'

statistics = {}

kept_beginning = []
kept_beginning_correctly = []
kept_middle = []
kept_middle_correctly = []
dropped_beginning = []
dropped_beginning_correctly = []
dropped_middle = []
dropped_middle_correctly = []

for doc_name, filterings in filtering_file.items():
    
#     if doc_name != 'doc-2.2':
#         continue
    
    print(doc_name.center(100, '-'))
    doc_gold = pet_reader.relations_dataset.GetSentencesWithIdsAndNerTagLabels(pet_reader.get_document_number(doc_name))
    
    for sentence_idx, tokens in enumerate(doc_gold):
        sentence_filterings = [f for f in filterings if f[0] == sentence_idx]
        for i, f in enumerate(sentence_filterings):
            t = f[1][0].lower()
            if t not in statistics:
                statistics[t] = {KEPT: 0, CORRECTLY_KEPT: 0, WRONGLY_KEPT: 0, DROPPED: 0, CORRECTLY_DROPPED: 0, WRONGLY_DROPPED: 0}
            print(' '.join([t[0] for t in tokens]))
            # tuple in token
            if tuple(f[1]) in tokens:
                if f[3] == 'kept':
                    statistics[t][KEPT] += 1
                    statistics[t][CORRECTLY_KEPT] += 1
                    print(sentence_idx, tokens[f[1][1]])
                    print(f, "CORRECTLY_KEPT\n")
#                     if t == 'or':
#                         print(sentence_idx, tokens)
#                         print(f, "CORRECTLY_KEPT")
                else:
                    statistics[t][DROPPED] += 1
                    statistics[t][WRONGLY_DROPPED] += 1
                    print(sentence_idx, tokens[f[1][1]])
                    print(f, "WRONGLY_DROPPED\n")
            # tuple not in token
            else:
                if f[3] == 'dropped':
                    statistics[t][DROPPED] += 1
                    statistics[t][CORRECTLY_DROPPED] += 1
                    print(sentence_idx, tokens[f[1][1]])
                    print(f, "CORRECTLY_DROPPED\n")
                else:
                    statistics[t][KEPT] += 1
                    statistics[t][WRONGLY_KEPT] += 1
                    print(sentence_idx, tokens[f[1][1]])
                    print(f, "WRONGLY_KEPT\n")
                    
            # kept/dropped in beginning/middle
            if i == 0:
                if f[3] == 'kept':
                    kept_beginning.append(f)
                    if tuple(f[1]) in tokens:
                        kept_beginning_correctly.append(f)
                else:
                    dropped_beginning.append(f)
                    if not tuple(f[1]) in tokens:
                        dropped_beginning_correctly.append(f)
            else:
                if f[3] == 'kept':
                    kept_middle.append(f)
                    if tuple(f[1]) in tokens:
                        kept_middle_correctly.append(f)
                else:
                    dropped_middle.append(f)
                    if not tuple(f[1]) in tokens:
                        dropped_middle_correctly.append(f)


----------------------------------------------doc-8.1-----------------------------------------------
The process finishes when the post was filled , precisely through the conclusion of a contract of employment .
2 ('when', 3, 'O')
[2, ['when', 3, 'B-XOR Gateway'], 2, 'kept'] WRONGLY_KEPT

----------------------------------------------doc-1.2-----------------------------------------------
If the customer decides that the costs are acceptable , the process continues , otherwise she takes her computer home unrepaired .
1 ('If', 0, 'B-XOR Gateway')
[1, ['If', 0, 'B-XOR Gateway'], 2, 'kept'] CORRECTLY_KEPT

If the customer decides that the costs are acceptable , the process continues , otherwise she takes her computer home unrepaired .
1 ('otherwise', 14, 'B-XOR Gateway')
[1, ['otherwise', 14, 'B-XOR Gateway'], 7, 'dropped'] WRONGLY_DROPPED

If an error is detected another arbitrary repair activity is executed , otherwise the repair is finished .
5 ('If', 0, 'B-XOR Gateway')
[5, ['If', 0, '

When the mail is received by the Registry , it is opened and sorted into groups for distribution , and thus registered in a manual incoming Mail Register .
3 ('When', 0, 'O')
[3, ['When', 0, 'B-XOR Gateway'], 2, 'kept'] WRONGLY_KEPT

If the mail is not compliant , a list of requisition explaining the reason for rejection is compiled and sent back to the party .
5 ('If', 0, 'B-XOR Gateway')
[5, ['If', 0, 'B-XOR Gateway'], 2, 'kept'] CORRECTLY_KEPT

Otherwise , the matter details ( types of action ) are captured and provided to the Cashier , who takes the applicable fees attached to the mail .
6 ('Otherwise', 0, 'B-XOR Gateway')
[6, ['Otherwise', 0, 'B-XOR Gateway'], 2, 'kept'] CORRECTLY_KEPT

----------------------------------------------doc-7.1-----------------------------------------------
If someone is interested , he sends a note to the Manager .
4 ('If', 0, 'B-XOR Gateway')
[4, ['If', 0, 'B-XOR Gateway'], 2, 'kept'] CORRECTLY_KEPT

Otherwise , he calls the next customer .
6 ('Other

An employee purchases a product or service he requires .
0 ('or', 5, 'I-Activity Data')
[0, ['or', 5, 'B-XOR Gateway'], 5, 'dropped'] CORRECTLY_DROPPED

A supervisor reviews the expense report and approves or rejects the report .
3 ('or', 8, 'I-Activity')
[3, ['or', 8, 'B-XOR Gateway'], 1, 'dropped'] CORRECTLY_DROPPED

Since the company has expense rules , there are circumstances where the supervisor can accept or reject the report upon first inspection .
4 ('or', 15, 'O')
[4, ['or', 15, 'B-XOR Gateway'], 1, 'dropped'] CORRECTLY_DROPPED

If the supervisor rejects the report , the employee , who submitted it , is given a chance to edit it , for example to correct errors or better describe an expense .
6 ('If', 0, 'B-XOR Gateway')
[6, ['If', 0, 'B-XOR Gateway'], 2, 'kept'] CORRECTLY_KEPT

If the supervisor rejects the report , the employee , who submitted it , is given a chance to edit it , for example to correct errors or better describe an expense .
6 ('or', 27, 'O')
[6, ['or', 27, 'B-

Of course , asking the customer whether he is generally interested is also important .
6 ('whether', 6, 'O')
[6, ['whether', 6, 'B-XOR Gateway'], 8, 'dropped'] CORRECTLY_DROPPED

If this is not the case , we leave him alone , except if the potential project budget is huge .
7 ('If', 0, 'B-XOR Gateway')
[7, ['If', 0, 'B-XOR Gateway'], 4, 'dropped'] WRONGLY_DROPPED

If this is not the case , we leave him alone , except if the potential project budget is huge .
7 ('if', 13, 'B-XOR Gateway')
[7, ['if', 13, 'B-XOR Gateway'], 4, 'dropped'] WRONGLY_DROPPED

If the customer is interested in the end , the next step is a detailed online presentation .
9 ('If', 0, 'B-XOR Gateway')
[9, ['If', 0, 'B-XOR Gateway'], 2, 'kept'] CORRECTLY_KEPT

It is given either by a sales representative or by a pre-sales employee in case of a more technical presentation .
10 ('either', 3, 'O')
[10, ['either', 3, 'B-XOR Gateway'], 5, 'dropped'] CORRECTLY_DROPPED

It is given either by a sales representative or by a pr

The process of an Office Supply Request starts when any employee of the organization submits an office supply request .
0 ('when', 8, 'O')
[0, ['when', 8, 'B-XOR Gateway'], 2, 'kept'] WRONGLY_KEPT

The supervisor must approve or ask for a change , or reject the request .
2 ('or', 4, 'I-Activity')
[2, ['or', 4, 'B-XOR Gateway'], 7, 'dropped'] CORRECTLY_DROPPED

The supervisor must approve or ask for a change , or reject the request .
2 ('or', 10, 'I-Activity')
[2, ['or', 10, 'B-XOR Gateway'], 7, 'dropped'] CORRECTLY_DROPPED

If the request is rejected the process will end .
3 ('If', 0, 'B-XOR Gateway')
[3, ['If', 0, 'B-XOR Gateway'], 2, 'kept'] CORRECTLY_KEPT

If the request is asked to make a change then it is returned to the petitioner / employee who can review the comments for the change request .
4 ('If', 0, 'B-XOR Gateway')
[4, ['If', 0, 'B-XOR Gateway'], 2, 'kept'] CORRECTLY_KEPT

If the request is approved it will go to the purchase department that will be in charge of making quo

### Stats for single keywords

In [8]:
rows = []
for keyword, stats in statistics.items():
    print(stats)
    kept_ratio = stats[KEPT] / (stats[KEPT] + stats[DROPPED])
    dropped_ratio = stats[DROPPED] / (stats[KEPT] + stats[DROPPED])
    stats["kept/dropped"] = f"{round(kept_ratio, 2)}/{round(dropped_ratio, 2)}"
    print(keyword, stats)
    rows.append({**{"word": keyword}, **stats})

{'kept': 12, 'correctly_kept': 0, 'wrongly_kept': 12, 'dropped': 3, 'correctly_dropped': 3, 'wrongly_dropped': 0}
when {'kept': 12, 'correctly_kept': 0, 'wrongly_kept': 12, 'dropped': 3, 'correctly_dropped': 3, 'wrongly_dropped': 0, 'kept/dropped': '0.8/0.2'}
{'kept': 63, 'correctly_kept': 58, 'wrongly_kept': 5, 'dropped': 4, 'correctly_dropped': 1, 'wrongly_dropped': 3}
if {'kept': 63, 'correctly_kept': 58, 'wrongly_kept': 5, 'dropped': 4, 'correctly_dropped': 1, 'wrongly_dropped': 3, 'kept/dropped': '0.94/0.06'}
{'kept': 6, 'correctly_kept': 6, 'wrongly_kept': 0, 'dropped': 7, 'correctly_dropped': 1, 'wrongly_dropped': 6}
otherwise {'kept': 6, 'correctly_kept': 6, 'wrongly_kept': 0, 'dropped': 7, 'correctly_dropped': 1, 'wrongly_dropped': 6, 'kept/dropped': '0.46/0.54'}
{'kept': 0, 'correctly_kept': 0, 'wrongly_kept': 0, 'dropped': 64, 'correctly_dropped': 44, 'wrongly_dropped': 20}
or {'kept': 0, 'correctly_kept': 0, 'wrongly_kept': 0, 'dropped': 64, 'correctly_dropped': 44, 'wrongl

In [10]:
df = pd.DataFrame.from_dict(rows)
df.head(10)
df.to_excel(os.path.join(ROOT_DIR, "data/paper_stats/token_cls/tc_filtering_stats_phrases.xlsx"), index=False)

### Stats for Keeping/Dropping in Beginning/Middle

In [18]:
# kept_beginning = []
# kept_middle = []
# dropped_beginning = []
# dropped_middle = []

print("kept_beginning", len(kept_beginning))
print("kept_beginning_correctly", len(kept_beginning_correctly))
print("kept_middle", len(kept_middle))
print("kept_middle_correctly", len(kept_middle_correctly))
print("dropped_beginning", len(dropped_beginning))
print("dropped_middle", len(dropped_middle))


kept_beginning 82
kept_beginning_correctly 68
kept_middle 7
kept_middle_correctly 3
dropped_beginning 67
dropped_middle 55


In [16]:
# keeping rates
print("Beginning:", round((len(kept_beginning) / (len(kept_beginning) + len(dropped_beginning))), 2))
print("Middle:", round((len(kept_middle) / (len(kept_middle) + len(dropped_middle))), 2))

Beginning: 0.55
Middle: 0.11


In [28]:
# correct rates
kept_beginning_correcty_percent = round((len(kept_beginning_correctly) / (len(kept_beginning) + len(kept_beginning_correctly))), 2)
kept_middle_correcty_percent = round((len(kept_middle_correctly) / (len(kept_middle) + len(kept_middle_correctly))), 2)
print(f"Kept beginning correctly: {kept_beginning_correcty_percent} from {len(kept_beginning)}")
print(f"Kept middle correctly: {kept_middle_correcty_percent} from {len(kept_middle)}")

print()

dropped_beginning_correcty_percent = round((len(dropped_beginning_correctly) / (len(dropped_beginning) + len(dropped_beginning_correctly))), 2)
dropped_middle_correcty_percent = round((len(dropped_middle_correctly) / (len(dropped_middle) + len(dropped_middle_correctly))), 2)
print(f"Dropped beginning correctly: {dropped_beginning_correcty_percent} from {len(dropped_beginning)}")
print(f"Dropped middle correctly: {dropped_middle_correcty_percent} from {len(dropped_middle)}")

Kept beginning correctly: 0.45 from 82
Kept middle correctly: 0.3 from 7

Dropped beginning correctly: 0.39 from 67
Dropped middle correctly: 0.4 from 55
